In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [ ]:
# Lee egresos hospitalarios y consultas ambulatorias
df_egresos = pd.read_csv("../data/processed/df_procesada.csv")
df_consultas = pd.read_csv(
    "../data/external/datos_limpios_diagnosticos.csv", encoding="latin-1", sep=";"
)

## Obtención de pacientes ambulatorios que tuvieron una hospitalización

In [ ]:
# Obtiene los pacientes de ambulatorio, que hayan aparecido alguna vez en hospitalizados
pacientes_amb_con_hosp = df_consultas.merge(
    df_egresos, how="left", left_on=["id_paciente", "ano"], right_on=["RUT", "ANO_EGR"]
)

# Calcula la cantidad de pacientes ambulatorios totales, y los que aparecieron en hospitalizados
conteo_de_pacientes_ambulatorios_con_hosp = pacientes_amb_con_hosp.groupby(
    ["ano", "nombre_especialidad"]
).agg(
    n_pacientes_ambulatorios=("id_paciente", "nunique"),
    n_pacientes_que_estuvieron_en_hosp=("RUT", "nunique"),
)